<a href="https://colab.research.google.com/github/rockey1006/verification/blob/main/DSP_verification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from zipfile import ZipFile
with ZipFile('//content/drive/MyDrive/batchwrong.zip', 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall('batchwrong')

In [3]:
from zipfile import ZipFile
with ZipFile('/content/drive/MyDrive/flipping.zip', 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall('flipping')

In [ ]:
!pip install deepface


In [5]:
from zipfile import ZipFile
with ZipFile('/content/drive/My Drive/facenet_weights.zip', 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall('/root/.deepface/weights/')

In [6]:

import sys
import base64
import cv2
import time
from deepface import DeepFace
from deepface.commons import functions, realtime, distance as dst
from deepface.basemodels import VGGFace, OpenFace, Facenet, FbDeepFace, DeepID
import numpy as np
import imageio

In [7]:
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
facenet_model = Facenet.loadModel()

def load_img2(img, do_random_crop, do_random_flip, image_size, do_prewhiten=True):
	images = np.zeros((1, image_size, image_size, 3))
	# scale pixel values
	face_pixels = img.astype('float32')
	# standardize pixel values across channels (global)
	mean, std = face_pixels.mean(), face_pixels.std()
	face_pixels = (face_pixels - mean) / std
	# transform face into one sample
	images[:,:,:,:] = face_pixels
	return images

In [8]:
def verify_distance(img1, img2):   # assumes image is in 160x160 format
	img1 = load_img2(img1, False, False, 160)
	img2 = load_img2(img2, False, False, 160)
	img1_representation = facenet_model.predict(img1)[0,:]
	img2_representation = facenet_model.predict(img2)[0,:]
	#distance1 = dst.findCosineDistance(img1_representation, img2_representation)
	#distance2 = dst.findEuclideanDistance(img1_representation, img2_representation)
	distance3 = dst.findEuclideanDistance(dst.l2_normalize(img1_representation), dst.l2_normalize(img2_representation))
	#result = str(distance1) + ' ' + str(distance2) + ' ' + str(distance3)
	return distance3
from skimage import io

In [9]:
def facedetect_image(img_file):  # detects face and returns 160x160 image around face
  imagePath = img_file
  image = io.imread(imagePath)
  margin = 3  # 3 pixel extra around detected face
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  faces = faceCascade.detectMultiScale(
		gray,
		scaleFactor=1.3,
		minNeighbors=3,
		minSize=(30, 30)
	)
 
  for (x, y, w, h) in faces:
		#cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    roi_color = image[y-margin:y + h+margin, x-margin:x + w+margin]
    dsize=(160,160)
    global imface
    imface = cv2.resize(roi_color, dsize, interpolation = cv2.INTER_AREA)
		#print("[INFO] Object found. Saving locally.")
		#cv2.imwrite(str(w) + str(h) + '_faces.jpg', imface1)
    encoded_string = imface 
    break
  return imface

In [76]:
import os 
srcPath = "/content/batchwrong/content/batchwrong/batch002/"
dstPath =  "/content/flipping/content/dsp_flipping2/batch002/" 
count=0
for name_file in os.listdir(srcPath):
    keyword=name_file
    print(keyword)
    img1 = facedetect_image(os.path.join(srcPath+str(keyword)))
    for name_file1 in os.listdir(dstPath):
        if keyword in name_file1:
          img2 = facedetect_image(os.path.join(dstPath+str(keyword)))
          res = verify_distance(img1,img2)
          if res<1.1:
            print("euclidean_l2 distance: ",res)
            count+=1

          
print(count)          

Marc_Grossman_0004_0.png
euclidean_l2 distance:  0.2851014
Susilo_Bambang_Yudhoyono_0001_0.png
euclidean_l2 distance:  0.16635896
Cruz_Bustamante_0001_0.png
euclidean_l2 distance:  0.20600183
Chen_Shui-bian_0003_0.png
Jerry_Regier_0002_0.png
Matthew_Broderick_0003_0.png
euclidean_l2 distance:  0.29530117
Mike_Brey_0001_0.png
euclidean_l2 distance:  0.1671121
Lauren_Hutton_0001_0.png
James_Gandolfini_0001_0.png
William_Hochul_0002_0.png
euclidean_l2 distance:  0.23845463
Ray_Nagin_0001_0.png
euclidean_l2 distance:  0.2250897
William_Bratton_0002_0.png
euclidean_l2 distance:  0.234239
Martha_Stewart_0002_0.png
Frank_Cassell_0001_0.png
John_Jumper_0002_0.png
Pete_Carroll_0002_0.png
Goldie_Hawn_0005_0.png
euclidean_l2 distance:  0.20400797
Heather_Mills_0001_0.png
euclidean_l2 distance:  0.10256031
Jose_Dirceu_0001_0.png
euclidean_l2 distance:  0.21356234
Isaiah_Washington_0001_0.png
euclidean_l2 distance:  0.30773118
Ashanti_0003_0.png
euclidean_l2 distance:  0.20522717
Patrick_McEnroe_00

In [65]:
def main():
  import os 
  srcPath = "/content/batchwrong/content/batchwrong/batch002/"
  dstPath =  "/content/flipping/content/dsp_flipping2/batch002/" 
  g = os.walk(r"/content/batchwrong/content/batchwrong/batch002/" )
  h = os.walk(r"/content/flipping/content/dsp_flipping2/batch002/" ) 
  for path1,dir_list2,file_list1 in g:  
    for name_list1 in file_list1:
      keyword= name_list1
      #print(name_list)
      #print(os.path.join(srcPath, name_list))
      img1 = facedetect_image(os.path.join(path1+str(name_list1)))
      for path2,dir_list2,file_list2 in h:  
        for name_list2 in file_list2:
          if name_list2 in name_list1:
            print(name_list2)
            
            img2 = facedetect_image(os.path.join(path2+str(name_list2)))
            res = verify_distance(img1,img2)
            #if res<1.2:
            print("euclidean_l2 distance: ",res)

                #print(path)
               #path1 = os.path.join(path,file_name)
               #path2 = os.path.join(srcPath+'/'+str(res))
               #with open(path1, 'rb') as rstream:
                 #container = rstream.read()
                 #with open(path2, 'wb') as wstream:
                   #wstream.write(container)

if __name__ == "__main__":
    sys.exit(int(main() or 0))

Marc_Grossman_0004_0.png
euclidean_l2 distance:  0.2851014


SystemExit: ignored